<a href="https://colab.research.google.com/github/rselent/lambda-buildweek2-asteroids/blob/master/build2_Asteroids_predicting_PHAs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# I don't know how replicable this is per user, but for now this should let me
# access my CSV, which is stored on Google Drive instead of github due to size
# (currently >100mb, maybe less after it's cleaned)

# If I perchance get it under 100mb, I'll migrate it over to github, but that
# won't be a primary (nor secondary) objective

from google.colab import drive
drive.mount( '/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install catboost
    !pip install ipywidgets
    !pip install lightgbm --install-option=--gpu        # Alternative to catboost,
                                                        # just in case..

In [53]:
import pandas as pd
import numpy as np

# New dataset, self-generated from JPL Small-Body Database Search Engine, 1/6/2020, 5:10pm
df = pd.read_csv( 'drive/My Drive/Colab Notebooks/Lambda School DS/Unit 2/Build Data/Asteroids_2020.csv', low_memory= False)
# Original dataset, found on Kaggle, generated from same source (JPL) but by a 3rd party, unknown date
df_old = pd.read_csv( 'drive/My Drive/Colab Notebooks/Lambda School DS/Unit 2/Build Data/Asteroid.csv', low_memory= False)

# df_old will be deprecated by week's end to reduce end filesize, but keeping for now (along with appropriate notations) to show process
df_old.shape, df.shape

((839714, 31), (921901, 34))

In [0]:
# I pulled the same datapoints / features from the original set, changed 1 (name -> full_name), and added 3 more:
# pdes - primary designation (more complete name list, presumably/just-in-case),
# and H_sigma and diameter_sigma - 1-sigma uncertainty values
# It seems ~80k more objects had been identified since the Kaggle set was generated and mine, also :o

df.head(20)

,full_name,pdes,a,e,i,om,w,q,ad,per_y,data_arc,condition_code,n_obs_used,H,H_sigma,neo,pha,diameter,diameter_sigma,extent,albedo,rot_per,GM,BV,UB,IR,spec_B,spec_T,G,moid,class,n,per,ma
0,1 Ceres,1,2.769165,0.076009,10.594067,80.305532,73.597694,2.558684,2.979647,4.608202,8822.0,0,1002,3.34,NaN,N,N,939.4,0.200,964.4 x 964.2 x 891.8,0.0900,9.074170,62.6284,0.713,0.426,NaN,C,G,0.12,1.594780,MBA,0.213885,1683.145708,77.372096
1,2 Pallas,2,2.772466,0.230337,34.836234,173.080063,310.048857,2.133865,3.411067,4.616444,72318.0,0,8490,4.13,NaN,N,N,545,18.000,582x556x500,0.1010,7.813200,14.3000,0.635,0.284,NaN,B,B,0.11,1.233240,MBA,0.213503,1686.155999,59.699133
2,3 Juno,3,2.669150,0.256942,12.988918,169.852756,248.138627,1.983332,3.354967,4.360814,72901.0,0,7109,5.33,NaN,N,N,246.596,10.594,NaN,0.2140,7.210000,NaN,0.824,0.433,NaN,Sk,S,0.32,1.034540,MBA,0.226019,1592.787286,34.925016
3,4 Vesta,4,2.361418,0.088721,7.141771,103.810804,150.728541,2.151909,2.570926,3.628837,24288.0,0,9325,3.20,NaN,N,N,525.4,0.200,572.6 x 557.2 x 446.4,0.4228,5.342128,17.8000,0.782,0.492,NaN,V,V,0.32,1.139480,MBA,0.271609,1325.432765,95.861936
4,5 Astraea,5,2.574249,0.191095,5.366988,141.576605,358.687607,2.082324,3.066174,4.130323,63519.0,0,2926,6.85,NaN,N,N,106.699,3.140,NaN,0.2740,16.806000,NaN,0.826,0.411,NaN,S,S,NaN,1.095890,MBA,0.238632,1508.600458,282.366289
5,6 Hebe,6,2.425160,0.203007,14.737901,138.640203,239.807490,1.932835,2.917485,3.776755,62329.0,0,6034,5.71,NaN,N,N,185.18,2.900,NaN,0.2679,7.274500,NaN,0.822,0.399,NaN,S,S,0.24,0.973965,MBA,0.260972,1379.459705,86.197923
6,7 Iris,7,2.385334,0.231206,5.523651,259.563231,145.265106,1.833831,2.936837,3.684105,62452.0,0,5206,5.51,NaN,N,N,199.83,10.000,NaN,0.2766,7.139000,NaN,0.855,0.484,NaN,S,S,NaN,0.846100,MBA,0.267535,1345.619196,140.419656
7,8 Flora,8,2.201764,0.156499,5.886955,110.889330,285.287462,1.857190,2.546339,3.267115,62655.0,0,2744,6.49,NaN,N,N,147.491,1.025,NaN,0.2260,12.865000,NaN,0.885,0.489,NaN,NaN,S,0.28,0.874176,MBA,0.301681,1193.313717,194.882895
8,9 Metis,9,2.385637,0.123114,5.576816,68.908577,6.417369,2.091931,2.679342,3.684806,61821.0,0,2649,6.28,NaN,N,N,190,NaN,NaN,0.1180,5.079000,NaN,0.858,0.496,NaN,NaN,S,0.17,1.106910,MBA,0.267484,1345.875362,276.861623
9,10 Hygiea,10,3.141539,0.112461,3.831560,283.202165,312.315210,2.788240,3.494839,5.568291,62282.0,0,3442,5.43,NaN,N,N,407.12,6.800,NaN,0.0717,27.630000,7.0000,0.696,0.351,NaN,C,C,NaN,1.778390,MBA,0.177007,2033.818281,152.184851


In [0]:
df.isnull().sum()

full_name              0
pdes                   0
a                      0
e                      0
i                      0
om                     0
w                      0
q                      0
ad                     5
per_y                  1
data_arc           38811
condition_code       901
n_obs_used             0
H                   2323
H_sigma           734362
neo                    5
pha                39715
diameter          784793
diameter_sigma    784921
extent            921883
albedo            785978
rot_per           903105
GM                921887
BV                920880
UB                920922
IR                921900
spec_B            920235
spec_T            920921
G                 921782
moid               39715
class                  0
n                      0
per                    5
ma                     1
dtype: int64

In [0]:
# Sigma features, while showing interesting/more precise data (or just precision
# metrics?), do not seem to be worth pursuing re: my objective. There are too many
# missing values in 'H_sigma', for example. 
# The delta between 'diameter' and 'diameter_sigma' is not all that large, however,
# so that could be an interesting secondary or perhaps tertiary objective.

# That said, right off the bat I will be dropping all features with more than 50%
# nan values, as not only is that just straight prohibitive to making any sort of 
# predictions of value, they also, coincidentally, happen to be the most 
# inconsequential features of the dataset (for my needs, and from my knowledge)

In [133]:
df1 = df.drop( columns= {'full_name', 'pdes', 'data_arc', 'H_sigma', 'diameter',
                         'diameter_sigma', 'extent', 'albedo', 'rot_per',
                         'GM', 'BV', 'UB', 'IR', 'spec_B', 'spec_T', 'G'})
df1.shape

(921901, 18)

In [0]:
df1.head()

,a,e,i,om,w,q,ad,per_y,condition_code,n_obs_used,H,neo,pha,moid,class,n,per,ma
0,2.769165,0.076009,10.594067,80.305532,73.597694,2.558684,2.979647,4.608202,0,1002,3.34,N,N,1.59478,MBA,0.213885,1683.145708,77.372096
1,2.772466,0.230337,34.836234,173.080063,310.048857,2.133865,3.411067,4.616444,0,8490,4.13,N,N,1.23324,MBA,0.213503,1686.155999,59.699133
2,2.669150,0.256942,12.988918,169.852756,248.138627,1.983332,3.354967,4.360814,0,7109,5.33,N,N,1.03454,MBA,0.226019,1592.787286,34.925016
3,2.361418,0.088721,7.141771,103.810804,150.728541,2.151909,2.570926,3.628837,0,9325,3.20,N,N,1.13948,MBA,0.271609,1325.432765,95.861936
4,2.574249,0.191095,5.366988,141.576605,358.687607,2.082324,3.066174,4.130323,0,2926,6.85,N,N,1.09589,MBA,0.238632,1508.600458,282.366289


In [0]:
df1.isnull().sum()

a                     0
e                     0
i                     0
om                    0
w                     0
q                     0
ad                    5
per_y                 1
condition_code      901
n_obs_used            0
H                  2323
neo                   5
pha               39715
moid              39715
class                 0
n                     0
per                   5
ma                    1
dtype: int64

In [0]:
# TODO: Everything?

# TODO2: Do 'pha' and 'moid' share the same Nans? <investigate>

# TODO3: Maybe split those observations, make them an ultimate test/prediction case? Or at least part of? Will that work?

# TODO4: I don't want to impute any of these Nan values; is there an algorithm/model that can accommodate? LightGBM? CatBoost?

# TODO5: <todo>

In [26]:
# Testing whether 'pha' contains the same NaN elements as 'moid':

dfTest1 = df1[[ 'a', 'e', 'pha']]
dfTest2 = df1[[ 'a', 'e', 'moid']]

dfTest1[ dfTest1.isnull().any(axis=1)].equals( dfTest2[dfTest2.isnull().any(axis=1)] )

False

In [0]:
# Sooo, they're not the same. Ok. What now... <TODO2 complete>

## HOWEVER, when I pull the 'pha' NaNs out of the df, all the 'moid' NaNs disappear
## in a .isnull().sum() inspection o.O

## Either way....

In [0]:
###  Train / Test / Validation splitting  ###

# Splitting 'pha' NaNs away first, planning on using as ultimate prediction case, maybe?
# <TODO3 complete>
phaNans = df1[ df1['pha'].isnull()]
df1 = df1[ df1['pha'].notnull()]

###     IMPORTANT     ###
##  only ~2k observations left with NaN values in potentially crucial features (below).
##  I don't _want_ to drop those asteroids from the dataset, because __ALL IT TAKES IS ONE__
##  BUT... if they cause too many complications, I will come back and drop them.
##  Just un-comment and run:
#df1 = df1.dropna()

##  2 HOURS LATER: Uncommenting, for sake of sanity. 
##      It's only a project / "for fun" after all, right? 
##                      RIGHT??? D: lol

## 1 HOUR AFTER THAT: Recommenting. It didn't solve the problem that 
##      I thought it would. So un-pressing the Big Red Button... for now...
##
##      Some things will look a little confusing from here on down. 
##          You've been warned.

In [39]:
df1.isnull().sum()          # pre-.dropna()

a                    0
e                    0
i                    0
om                   0
w                    0
q                    0
ad                   5
per_y                1
condition_code      15
n_obs_used           0
H                 2322
neo                  5
pha                  0
moid                 0
class                0
n                    0
per                  5
ma                   1
dtype: int64

In [49]:
#####       BASELINE       #####
## (post NaN purge, pre .dropna() purge)

print( df1['pha'].value_counts())
print( df1['pha'].value_counts( normalize= True) * 100)     # *100 for % 

N    880160
Y      2026
Name: pha, dtype: int64
N    99.770343
Y     0.229657
Name: pha, dtype: float64


Baseline **Accuracy** is about 1/4th of a percent (0.23%), which means accuracy is basically meaningless for this, and I need to look at a more appropriate metric, like Recall, perhaps

In [132]:
#####       BASELINE       #####
##   (post .dropna() purge)

## Leaving this here for posterity and comparison's sake for now,
## but since I undid df1.dropna(), **THIS DOESN'T MATTER.**
## Ofc, this will be cleaned up if I decide above's .dropna() is unnecessary

print( df1['pha'].value_counts())
print( df1['pha'].value_counts( normalize= True) * 100)     # *100 for % 

0    877823
1      2025
Name: pha, dtype: int64
0    99.769847
1     0.230153
Name: pha, dtype: float64


In [0]:
# encode 'pha' and 'neo' vals from Y/N -> 1/0, respectively
# this could easily be a function instead, but for 2 instances? 
# doesn't seem worth
df1[ df1.columns[ 12]] = df1[ df1.columns[ 12]].replace( {'Y': 1, 'N': 0})
df1[ df1.columns[ 11]] = df1[ df1.columns[ 11]].replace( {'Y': 1, 'N': 0})

### Train/test/val split continued below. But first:

In [81]:
## (pre .dropna() purge)
## Leaving this for now, because it's interesting

df1[ df1.neo.isnull()]

,a,e,i,om,w,q,ad,per_y,condition_code,n_obs_used,H,neo,pha,moid,class,n,per,ma
723832,-6.149566e+00,1.855356,8.643584,269.988114,332.989886,5.260067,NaN,0.0,NaN,32,10.9000,NaN,0,4.250330,HYA,6.463048e-02,NaN,-7.070368e+01
903934,-1.272345e+00,1.201134,122.741706,24.596910,241.810536,0.255912,NaN,NaN,NaN,207,22.0800,NaN,0,0.095821,HYA,6.867469e-01,NaN,5.115762e+01
904072,-7.867348e+03,1.000816,142.636343,276.230354,326.002456,6.417541,NaN,0.0,NaN,423,10.6440,NaN,0,5.465020,HYA,1.412412e-06,NaN,-3.431440e-04
918299,-1.042792e+05,1.000056,138.380218,130.468071,287.870052,5.858483,NaN,0.0,NaN,21,13.4340,NaN,0,5.116310,HYA,2.926897e-08,NaN,1.081181e-05
919972,-4.304052e+06,1.000002,89.822516,300.385429,59.823007,8.827042,NaN,0.0,NaN,74,9.5567,NaN,0,8.275260,HYA,1.103794e-10,NaN,-5.664189e-08


In [87]:
df.iloc[ 903934]

full_name                'Oumuamua (A/2017 U1)
pdes                                   2017 U1
a                                     -1.27235
e                                      1.20113
i                                      122.742
om                                     24.5969
w                                      241.811
q                                     0.255912
ad                                         NaN
per_y                                      NaN
data_arc                                    80
condition_code                             NaN
n_obs_used                                 207
H                                        22.08
H_sigma                                0.44525
neo                                        NaN
pha                                          N
diameter                                   NaN
diameter_sigma                             NaN
extent                                     NaN
albedo                                     NaN
rot_per      

(Above)

**I was curious** about why, when I was (manually) encoding 'neo' from Y/N to 1/0, it would only encode as a float64 dtype, instead of the int64 dtype that 'pha' defaulted to. Upon investigating, this was because there were still NaN values in the 'neo' feature. "*Well, why are there NaNs? **What are** these NaNs?*"  
So, there are 5 of them, and in trying to figure out **why** they were NaN'ed as near-earth objects, I discovered that one of them in particular **DID** fit the classification of a near-earth object.  
Since I had names dropped from this modified subset, I had to do an .iloc lookup in the main dataset. Meanwhile, my brain is goind "*Did we just make a **discovery**??!?!*"

Turns out, this NaN'd-but-definitely-a-near-earth-object (according to measurements) was none other than *'Oumuamua*. An extra-solar/interstellar asteroid that was discovered in October 2017, which made headlines for being the first discovery of any such object ever, and the fact that it was so close to us here on Earth was also quite a shock.

*'Oumuamua*'s origins are unknown, along with how long it's been traveling through space. Since its discovery, though, another interstellar object was spotted in August 2019, comet *Borisov*, which was 14x larger than Earth and reached a perihelion (distance from the Sun) of approximately 2 AU.

In [136]:
####       FINALLY       ####
## train/test/validation split, For Real This Time
from sklearn.model_selection import train_test_split

dummyTrain, validate = train_test_split( df1, train_size= .7, 
                                        stratify= df1[ 'pha'], 
                                        random_state= 16)
train, test = train_test_split( dummyTrain, train_size= .7, 
                               stratify= dummyTrain[ 'pha'], 
                               random_state= 16)

print( train['pha'].value_counts() )
print( validate['pha'].value_counts() )
print( test['pha'].value_counts() )

0    431278
1       993
Name: pha, dtype: int64
0    264048
1       608
Name: pha, dtype: int64
0    184834
1       425
Name: pha, dtype: int64


In [0]:
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

target = 'pha'
features = train.columns.drop( [target])
# features = train.drop( columns= [target])             ## <-- THIS is causing 
                                                        ## problems with
xTrain = train[ features]                               ## <-- this. WHY??
yTrain = train[ target]
xValidate = validate[ features]
yValidate = validate[ target]
xTest = test[ features]



In [0]:
logit = LogisticRegression( solver= 'lbfgs', multi_class= 'auto',
                            n_jobs= -1, random_state= 16)
